In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
from imutils.video import VideoStream
import imutils

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\jagdi\Downloads\Mask/','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:/Users/jagdi/Downloads/Mask/','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxtPath
    

'C:\\Users\\jagdi\\Downloads\\Mask/\\deploy.prototxt'

In [4]:
weightsPath

'C:/Users/jagdi/Downloads/Mask/\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
net

< cv2.dnn.Net 000001AA63D96D10>

In [7]:
model=load_model(r'C:\Users\jagdi\Downloads\Mask\mobilenet_v2.model')

In [8]:
image=cv2.imread(r'C:\Users\jagdi\Downloads\Mask\Example\example\ex_01.png')


In [9]:
image

array([[[ 51,  46, 117],
        [ 51,  47, 118],
        [ 51,  46, 117],
        ...,
        [ 98, 105, 109],
        [ 99, 104, 108],
        [ 99, 104, 108]],

       [[ 51,  47, 118],
        [ 51,  47, 118],
        [ 51,  47, 118],
        ...,
        [ 83,  87,  94],
        [ 83,  83,  85],
        [ 87,  85,  86]],

       [[ 47,  44, 113],
        [ 48,  45, 114],
        [ 52,  48, 118],
        ...,
        [ 59,  60,  62],
        [ 54,  54,  56],
        [ 65,  67,  70]],

       ...,

       [[ 36,  29,  31],
        [ 40,  35,  38],
        [ 28,  22,  25],
        ...,
        [ 16,  14,  17],
        [ 16,  14,  17],
        [ 16,  14,  17]],

       [[ 45,  41,  43],
        [ 53,  40,  41],
        [ 28,  22,  25],
        ...,
        [ 25,  20,  23],
        [ 19,  16,  19],
        [ 16,  14,  17]],

       [[ 61,  45,  45],
        [ 56,  40,  41],
        [ 28,  22,  25],
        ...,
        [ 17,  15,  18],
        [ 16,  14,  17],
        [ 16,  14,  17]]

In [10]:
(h,w)=image.shape[:2]

In [11]:
(h,w)

(432, 768)

In [12]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))


In [13]:
blob

array([[[[ -53.,  -53.,  -53., ...,   14.,    1.,   -9.],
         [ -55.,  -50.,  -49., ...,  -13.,  -28.,  -38.],
         [ -61.,  -48.,  -48., ...,  -30.,  -42.,  -56.],
         ...,
         [ -67.,  -88.,  -71., ...,  -88.,  -88.,  -88.],
         [ -61.,  -87.,  -69., ...,  -88.,  -88.,  -87.],
         [ -48.,  -84.,  -68., ...,  -88.,  -88.,  -88.]],

        [[-130., -131., -131., ...,  -57.,  -68.,  -78.],
         [-132., -127., -126., ...,  -85.,  -99., -111.],
         [-137., -125., -126., ..., -102., -113., -130.],
         ...,
         [-147., -163., -151., ..., -163., -163., -163.],
         [-141., -163., -150., ..., -163., -163., -163.],
         [-136., -160., -149., ..., -163., -163., -163.]],

        [[  -5.,   -6.,   -6., ...,    0.,  -11.,  -20.],
         [  -8.,   -4.,   -3., ...,  -28.,  -40.,  -55.],
         [ -17.,   -8.,   -3., ...,  -42.,  -58.,  -72.],
         ...,
         [ -90., -106.,  -95., ..., -106., -106., -106.],
         [ -85., -106.,  -

In [14]:
blob.shape

(1, 3, 300, 300)

In [15]:
net.setInput(blob)
detections=net.forward()

In [16]:
detections

array([[[[0.        , 1.        , 0.9015393 , ..., 0.35069847,
          0.8943858 , 0.52013457],
         [0.        , 1.        , 0.8833362 , ..., 0.31085664,
          0.5540466 , 0.46340668],
         [0.        , 1.        , 0.79707986, ..., 0.4063273 ,
          0.4787544 , 0.5570959 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [17]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()